In [163]:
import nfl_data_py as nfl
import pandas as pd

In [164]:
# pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [165]:
df = nfl.import_pbp_data([2024])

2024 done.
Downcasting floats.


In [166]:
df

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,None,None,None,...,0.0,0.0,-0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,0.257819,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,61.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,-0.200602,NaN,NaN,NaN,NaN,NaN,0.456761,-45.676102
3,83.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,2.028874,1.345418,9.321221,8.0,0.509778,0.363807,0.576656,42.334431
4,108.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,0.754242,0.882798,5.783560,4.0,0.668478,0.255140,0.426443,57.355690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49311,4718.0,2024_21_WAS_PHI,2025012600,PHI,WAS,POST,21,WAS,away,PHI,...,0.0,0.0,-0.722550,NaN,NaN,NaN,NaN,NaN,0.949295,-94.929489
49312,4747.0,2024_21_WAS_PHI,2025012600,PHI,WAS,POST,21,WAS,away,PHI,...,0.0,0.0,-2.747494,NaN,NaN,NaN,NaN,NaN,0.930013,-93.001251
49313,4769.0,2024_21_WAS_PHI,2025012600,PHI,WAS,POST,21,PHI,home,WAS,...,0.0,0.0,-0.650122,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49314,4791.0,2024_21_WAS_PHI,2025012600,PHI,WAS,POST,21,PHI,home,WAS,...,0.0,0.0,-2.106240,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
df.loc[df.home_team=='BUF']

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,None,None,None,...,0.0,0.0,-0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,0.257819,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,61.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,-0.200602,NaN,NaN,NaN,NaN,NaN,0.456761,-45.676102
3,83.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,2.028874,1.345418,9.321221,8.0,0.509778,0.363807,0.576656,42.334431
4,108.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,0.754242,0.882798,5.783560,4.0,0.668478,0.255140,0.426443,57.355690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48405,3767.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,BUF,home,BAL,...,0.0,0.0,0.135715,NaN,NaN,NaN,NaN,NaN,0.056761,-5.676054
48406,3806.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,None,None,None,...,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48407,3794.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,BUF,home,BAL,...,0.0,0.0,-0.377236,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48408,3823.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,BUF,home,BAL,...,0.0,0.0,-1.757979,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
game = df.loc[df.game_id=="2024_20_BAL_BUF"]

In [169]:
game

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
48252,1.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,None,None,None,...,0.0,0.0,-0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48253,39.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,BAL,away,BUF,...,0.0,0.0,0.143454,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48254,62.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,BAL,away,BUF,...,0.0,0.0,-0.595957,NaN,NaN,NaN,NaN,NaN,0.468283,-46.828278
48255,84.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,BAL,away,BUF,...,0.0,0.0,-0.550132,NaN,NaN,NaN,NaN,NaN,0.663315,-66.331459
48256,106.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,BAL,away,BUF,...,0.0,0.0,1.911900,NaN,NaN,NaN,NaN,NaN,0.982585,1.741505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48405,3767.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,BUF,home,BAL,...,0.0,0.0,0.135715,NaN,NaN,NaN,NaN,NaN,0.056761,-5.676054
48406,3806.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,None,None,None,...,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48407,3794.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,BUF,home,BAL,...,0.0,0.0,-0.377236,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48408,3823.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,BUF,home,BAL,...,0.0,0.0,-1.757979,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
game.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 48252 to 48409
Columns: 372 entries, play_id to pass_oe
dtypes: float32(199), int32(7), int64(1), object(165)
memory usage: 333.3+ KB


In [171]:
game.columns.tolist()

['play_id',
 'game_id',
 'old_game_id',
 'home_team',
 'away_team',
 'season_type',
 'week',
 'posteam',
 'posteam_type',
 'defteam',
 'side_of_field',
 'yardline_100',
 'game_date',
 'quarter_seconds_remaining',
 'half_seconds_remaining',
 'game_seconds_remaining',
 'game_half',
 'quarter_end',
 'drive',
 'sp',
 'qtr',
 'down',
 'goal_to_go',
 'time',
 'yrdln',
 'ydstogo',
 'ydsnet',
 'desc',
 'play_type',
 'yards_gained',
 'shotgun',
 'no_huddle',
 'qb_dropback',
 'qb_kneel',
 'qb_spike',
 'qb_scramble',
 'pass_length',
 'pass_location',
 'air_yards',
 'yards_after_catch',
 'run_location',
 'run_gap',
 'field_goal_result',
 'kick_distance',
 'extra_point_result',
 'two_point_conv_result',
 'home_timeouts_remaining',
 'away_timeouts_remaining',
 'timeout',
 'timeout_team',
 'td_team',
 'td_player_name',
 'td_player_id',
 'posteam_timeouts_remaining',
 'defteam_timeouts_remaining',
 'total_home_score',
 'total_away_score',
 'posteam_score',
 'defteam_score',
 'score_differential',
 'po

In [172]:
game.tail()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
48405,3767.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,BUF,home,BAL,...,0.0,0.0,0.135715,NaN,NaN,NaN,NaN,NaN,0.056761,-5.676054
48406,3806.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,None,None,None,...,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48407,3794.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,BUF,home,BAL,...,0.0,0.0,-0.377236,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48408,3823.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,BUF,home,BAL,...,0.0,0.0,-1.757979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48409,3845.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,BUF,home,BAL,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
game.loc[df.play_id==1]

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
48252,1.0,2024_20_BAL_BUF,2025011901,BUF,BAL,POST,20,None,None,None,...,0.0,0.0,-0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
game.loc[[48253,48254], ['play_id','posteam','time','down','ydstogo','yrdln','play_type','desc','ep', 'epa','total_home_epa','total_away_epa']]

,play_id,posteam,time,down,ydstogo,yrdln,play_type,desc,ep,epa,total_home_epa,total_away_epa
48253,39.0,BAL,15:00,NaN,0.0,BUF 35,kickoff,2-T.Bass kicks 62 yards from BUF 35 to BAL 3. 34-K.Mitchell to BAL 27 for 24 yards (42-Do.Williams).,0.770222,0.143454,-0.143454,0.143454
48254,62.0,BAL,14:54,1.0,10.0,BAL 27,run,(14:54) 22-D.Henry left guard to BAL 28 for 1 yard (50-G.Rousseau).,0.913676,-0.595957,0.452503,-0.452503


In [175]:
game.loc[[48405], ['play_id','time','down','ydstogo','yrdln','play_type','desc','ep', 'epa','total_home_epa','total_away_epa']]

,play_id,time,down,ydstogo,yrdln,play_type,desc,ep,epa,total_home_epa,total_away_epa
48405,3767.0,01:32,1.0,10.0,BAL 47,run,(1:32) 70-A.Anderson reported in as eligible. 26-Ty.Johnson right guard to BAL 30 for 17 yards.,1.9995,0.135715,1.575348,-1.575348


In [176]:
game.tail(20)['desc']

48390                                                                                                                                                                                                                  Timeout #2 by BAL at 04:17.
48391                                                                                                                                                (4:17) (Shotgun) 17-J.Allen up the middle to BAL 2 for no gain (29-A.Washington; 98-T.Jones).
48392                                                                                                                                                                                                                  Timeout #1 by BUF at 03:31.
48393                                                                                                                                                         (3:31) 2-T.Bass 21 yard field goal is GOOD, Center-69-R.Ferguson, Holder-8-S.Martin.
48394                       

In [177]:
game.loc[[48403], ['play_id','time','down','ydstogo','yrdln','play_type','desc','ep', 'epa','total_home_epa','total_away_epa']]

,play_id,time,down,ydstogo,yrdln,play_type,desc,ep,epa,total_home_epa,total_away_epa
48403,3728.0,01:33,NaN,0.0,BUF 2,pass,TWO-POINT CONVERSION ATTEMPT. 8-L.Jackson pass to 89-M.Andrews is incomplete. ATTEMPT FAILS. ** Injury Update: BAL-7-R.Bateman has returned to the game.,0.947,-0.947,0.236817,-0.236817


In [178]:
game.loc[48409, ['total_home_epa', 'total_home_score','total_away_epa', 'total_away_score']]

total_home_epa     -0.559867
total_home_score        27.0
total_away_epa      0.559867
total_away_score        25.0
Name: 48409, dtype: object

In [179]:
game.groupby('posteam')['epa'].agg('sum')

posteam
BAL    9.064721
BUF    8.504854
Name: epa, dtype: float32

In [180]:
game[['time','down','ydstogo','yrdln','posteam','epa']]

,time,down,ydstogo,yrdln,posteam,epa
48252,15:00,NaN,0.0,BUF 35,None,-0.000000
48253,15:00,NaN,0.0,BUF 35,BAL,0.143454
48254,14:54,1.0,10.0,BAL 27,BAL,-0.595957
48255,14:18,2.0,9.0,BAL 28,BAL,-0.550132
48256,13:37,3.0,8.0,BAL 29,BAL,1.911900
...,...,...,...,...,...,...
48405,01:32,1.0,10.0,BAL 47,BUF,0.135715
48406,01:18,NaN,0.0,BAL 30,None,0.000000
48407,01:18,1.0,10.0,BAL 30,BUF,-0.377236
48408,00:38,2.0,11.0,BAL 31,BUF,-1.757979


In [181]:
game.loc[game['posteam']=='BAL', ['time','down','ydstogo','yrdln','posteam','epa','total_home_epa']]

,time,down,ydstogo,yrdln,posteam,epa,total_home_epa
48253,15:00,NaN,0.0,BUF 35,BAL,0.143454,-0.143454
48254,14:54,1.0,10.0,BAL 27,BAL,-0.595957,0.452503
48255,14:18,2.0,9.0,BAL 28,BAL,-0.550132,1.002636
48256,13:37,3.0,8.0,BAL 29,BAL,1.911900,-0.909264
48257,12:54,1.0,10.0,BAL 38,BAL,-0.518912,-0.390352
...,...,...,...,...,...,...,...
48399,02:14,2.0,7.0,BAL 26,BAL,1.868864,3.691315
48400,02:00,1.0,10.0,BUF 47,BAL,-0.425952,4.117267
48401,01:52,2.0,6.0,BUF 43,BAL,1.657467,2.459800
48402,01:44,1.0,10.0,BUF 24,BAL,3.169983,-0.710183


In [182]:
game.groupby(['posteam','play_type']).agg({'epa': 'sum'})

epa
posteam play_type            
BAL     extra_point  0.067573
        field_goal   1.364117
        kickoff      0.286451
        no_play     -0.769099
        pass        -0.689817
        qb_kneel     0.000000
        run          8.805497
BUF     extra_point  0.202719
        field_goal   1.922578
        kickoff      2.550836
        no_play      1.831737
        pass        -1.731060
        punt        -0.296110
        qb_kneel    -2.135215
        run          6.159369

In [183]:
game.loc[game['play_type']=='qb_kneel', ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep', 'epa']]

,posteam,time,qtr,down,ydstogo,yrdln,play_type,desc,ep,epa
48328,BAL,00:09,2.0,1.0,10.0,BAL 33,qb_kneel,(:09) 8-L.Jackson kneels to BAL 32 for -1 yards.,0.000000,0.000000
48407,BUF,01:18,4.0,1.0,10.0,BAL 30,qb_kneel,(1:18) 17-J.Allen kneels to BAL 31 for -1 yards.,2.135215,-0.377236
48408,BUF,00:38,4.0,2.0,11.0,BAL 31,qb_kneel,(:38) 17-J.Allen kneels to BAL 32 for -1 yards.,1.757979,-1.757979


In [184]:
game.loc[game['play_type']=='punt', ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep', 'epa']]

,posteam,time,qtr,down,ydstogo,yrdln,play_type,desc,ep,epa
48285,BUF,00:48,1.0,4.0,17.0,BUF 47,punt,"(:48) 8-S.Martin punts 44 yards to BAL 9, Center-69-R.Ferguson, fair catch by 18-S.Sims.",-0.399311,0.507688
48334,BUF,14:00,3.0,4.0,9.0,BUF 26,punt,"(14:00) 8-S.Martin punts 54 yards to BAL 20, Center-69-R.Ferguson. 18-S.Sims to BAL 36 for 16 yards (26-Ty.Johnson).",-1.338296,-0.250728
48352,BUF,05:30,3.0,4.0,5.0,BUF 48,punt,"(5:30) 8-S.Martin punts 52 yards to end zone, Center-69-R.Ferguson, Touchback.",0.100737,-0.553069


In [185]:
game.loc[game['drive']==4, ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa']]

,posteam,time,qtr,down,ydstogo,yrdln,play_type,desc,ep,epa
48279,BUF,03:52,1.0,1.0,10.0,BUF 43,pass,(3:52) (Shotgun) 17-J.Allen pass short middle to 86-D.Kincaid to BAL 46 for 11 yards (40-M.Harrison).,2.558003,0.608715
48280,BUF,03:09,1.0,1.0,10.0,BAL 46,run,(3:09) (Shotgun) 70-A.Anderson reported in as eligible. 4-J.Cook right guard to BAL 47 for -1 yards (95-T.Robinson).,3.166719,-0.810377
48281,BUF,02:27,1.0,2.0,11.0,BAL 47,no_play,"(2:27) (Shotgun) 17-J.Allen pass short right to 1-C.Samuel to BAL 39 for 8 yards (49-C.Board; 2-N.Wiggins). PENALTY on BUF-73-D.Dawkins, Offensive Holding, 10 yards, enforced at BAL 47 - No Play.",2.356342,-0.623418
48282,BUF,02:03,1.0,2.0,21.0,BUF 43,no_play,"(2:03) (Shotgun) PENALTY on BAL-99-O.Oweh, Neutral Zone Infraction, 5 yards, enforced at BUF 43 - No Play.",1.732925,0.224084
48283,BUF,01:39,1.0,2.0,16.0,BUF 48,pass,(1:39) (Shotgun) 17-J.Allen sacked at BUF 47 for -1 yards (sack split by 53-K.Van Noy and 92-N.Madubuike).,1.957008,-1.071717
48284,BUF,00:53,1.0,3.0,17.0,BUF 47,pass,(:53) (Shotgun) 17-J.Allen pass incomplete deep right to 88-D.Knox [53-K.Van Noy].,0.885292,-1.284603
48285,BUF,00:48,1.0,4.0,17.0,BUF 47,punt,"(:48) 8-S.Martin punts 44 yards to BAL 9, Center-69-R.Ferguson, fair catch by 18-S.Sims.",-0.399311,0.507688


In [186]:
game.groupby(['posteam', 'down']).agg({'ydstogo': 'mean'})

ydstogo
posteam down          
BAL     1.0   9.551724
        2.0   7.941176
        3.0   6.090909
        4.0   7.000000
BUF     1.0   9.730769
        2.0   7.347826
        3.0   5.416667
        4.0   7.166667

In [187]:
game.groupby(['posteam', 'down']).agg({'epa': 'mean'})

epa
posteam down          
BAL     1.0   0.305229
        2.0  -0.592140
        3.0   0.774765
        4.0   1.099010
BUF     1.0   0.016026
        2.0  -0.048122
        3.0   0.188344
        4.0   0.696882

In [188]:
game.groupby('drive').agg({'epa': 'sum'})

,epa
drive,
1.0,6.297351
2.0,5.636252
3.0,-3.436857
4.0,-2.449626
5.0,-4.354531
6.0,2.604665
7.0,1.948084
8.0,6.183342
9.0,-0.061913


In [189]:
game.loc[game['drive']==1, ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa']]

,posteam,time,qtr,down,ydstogo,yrdln,play_type,desc,ep,epa
48253,BAL,15:00,1.0,NaN,0.0,BUF 35,kickoff,2-T.Bass kicks 62 yards from BUF 35 to BAL 3. 34-K.Mitchell to BAL 27 for 24 yards (42-Do.Williams).,0.770222,0.143454
48254,BAL,14:54,1.0,1.0,10.0,BAL 27,run,(14:54) 22-D.Henry left guard to BAL 28 for 1 yard (50-G.Rousseau).,0.913676,-0.595957
48255,BAL,14:18,1.0,2.0,9.0,BAL 28,run,"(14:18) (Shotgun) 22-D.Henry left tackle to BAL 29 for 1 yard (57-A.Epenesa, 92-Da.Jones).",0.317719,-0.550132
48256,BAL,13:37,1.0,3.0,8.0,BAL 29,run,(13:37) (Shotgun) 8-L.Jackson scrambles left tackle to BAL 38 for 9 yards (7-Ta.Johnson).,-0.232413,1.911900
48257,BAL,12:54,1.0,1.0,10.0,BAL 38,pass,(12:54) (Shotgun) 8-L.Jackson pass incomplete short left to 22-D.Henry.,1.679486,-0.518912
48258,BAL,12:47,1.0,2.0,10.0,BAL 38,pass,(12:47) (Shotgun) 8-L.Jackson pass deep right to 80-I.Likely to BUF 23 for 39 yards (47-C.Benford).,1.160574,3.298342
48259,BAL,12:00,1.0,1.0,10.0,BUF 23,run,(12:00) (Shotgun) 22-D.Henry left tackle to BUF 21 for 2 yards (57-A.Epenesa; 98-A.Johnson).,4.458917,-0.448721
48260,BAL,11:15,1.0,2.0,8.0,BUF 21,pass,(11:15) (Shotgun) 8-L.Jackson pass short right to 80-I.Likely to BUF 16 for 5 yards (7-Ta.Johnson).,4.010196,0.037648
48261,BAL,10:46,1.0,3.0,3.0,BUF 16,pass,"(10:46) (No Huddle, Shotgun) 8-L.Jackson pass short middle to 7-R.Bateman for 16 yards, TOUCHDOWN.",4.047844,2.952156
48262,BAL,10:40,1.0,NaN,0.0,BUF 15,extra_point,"9-J.Tucker extra point is GOOD, Center-46-N.Moore, Holder-11-J.Stout.",0.932427,0.067573


In [190]:
game.loc[game['play_type']=='kickoff', ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa']]

,posteam,time,qtr,down,ydstogo,yrdln,play_type,desc,ep,epa
48253,BAL,15:00,1.0,NaN,0.0,BUF 35,kickoff,2-T.Bass kicks 62 yards from BUF 35 to BAL 3. 34-K.Mitchell to BAL 27 for 24 yards (42-Do.Williams).,0.770222,0.143454
48263,BUF,10:40,1.0,NaN,0.0,BAL 35,kickoff,"9-J.Tucker kicks 65 yards from BAL 35 to end zone, Touchback to the BUF 30.",1.431321,0.327765
48276,BAL,04:43,1.0,NaN,0.0,BUF 35,kickoff,2-T.Bass kicks 63 yards from BUF 35 to BAL 2. 34-K.Mitchell to BAL 28 for 26 yards (41-R.Gilliam; 42-Do.Williams).,0.878854,0.119256
48301,BAL,08:52,2.0,NaN,0.0,BUF 35,kickoff,2-T.Bass kicks 59 yards from BUF 35 to BAL 6. 34-K.Mitchell pushed ob at BAL 35 for 29 yards (47-C.Benford).,1.051915,0.408695
48312,BUF,03:43,2.0,NaN,0.0,BAL 35,kickoff,"11-J.Stout kicks 65 yards from BAL 35 to end zone, Touchback to the BUF 30.",0.884231,0.365796
48327,BAL,00:16,2.0,NaN,0.0,BUF 35,kickoff,2-T.Bass kicks 64 yards from BUF 35 to BAL 1. 34-K.Mitchell to BAL 33 for 32 yards (46-J.Ingram).,0.061913,-0.061913
48330,BUF,15:00,3.0,NaN,0.0,BAL 35,kickoff,11-J.Stout kicks 65 yards from BAL 35 to BUF 0. 22-R.Davis to BUF 25 for 25 yards (54-K.Welch; 88-C.Kolar).,1.474098,0.010567
48345,BUF,09:47,3.0,NaN,0.0,BAL 35,kickoff,"11-J.Stout kicks 65 yards from BAL 35 to end zone, Touchback to the BUF 30.",1.403757,0.329688
48361,BUF,01:37,3.0,NaN,0.0,BAL 35,kickoff,"11-J.Stout kicks 65 yards from BAL 35 to end zone, Touchback to the BUF 30.",1.355946,0.314205
48373,BAL,12:04,4.0,NaN,0.0,BUF 35,kickoff,"2-T.Bass kicks 65 yards from BUF 35 to end zone, Touchback to the BAL 30.",0.996922,0.263449


In [196]:
game.loc[game['desc']=='END GAME', ['home_team','total_home_score','total_home_epa','total_home_rush_epa','total_home_pass_epa','away_team','total_away_score','total_away_epa','total_away_rush_epa','total_away_pass_epa']]

,home_team,total_home_score,total_home_epa,total_home_rush_epa,total_home_pass_epa,away_team,total_away_score,total_away_epa,total_away_rush_epa,total_away_pass_epa
48409,BUF,27.0,-0.559867,-2.646128,-1.041243,BAL,25.0,0.559867,2.646128,1.041243


In [199]:
game.loc[game['play_type']=='run', ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']]

,posteam,time,qtr,down,ydstogo,yrdln,play_type,desc,ep,epa,total_home_epa,total_away_epa
48254,BAL,14:54,1.0,1.0,10.0,BAL 27,run,(14:54) 22-D.Henry left guard to BAL 28 for 1 yard (50-G.Rousseau).,0.913676,-0.595957,0.452503,-0.452503
48255,BAL,14:18,1.0,2.0,9.0,BAL 28,run,"(14:18) (Shotgun) 22-D.Henry left tackle to BAL 29 for 1 yard (57-A.Epenesa, 92-Da.Jones).",0.317719,-0.550132,1.002636,-1.002636
48256,BAL,13:37,1.0,3.0,8.0,BAL 29,run,(13:37) (Shotgun) 8-L.Jackson scrambles left tackle to BAL 38 for 9 yards (7-Ta.Johnson).,-0.232413,1.911900,-0.909264,0.909264
48259,BAL,12:00,1.0,1.0,10.0,BUF 23,run,(12:00) (Shotgun) 22-D.Henry left tackle to BUF 21 for 2 yards (57-A.Epenesa; 98-A.Johnson).,4.458917,-0.448721,-3.239973,3.239973
48265,BUF,10:35,1.0,2.0,10.0,BUF 30,run,(10:35) (Shotgun) 4-J.Cook up the middle to BUF 32 for 2 yards (98-T.Jones; 92-N.Madubuike).,1.207955,-0.726229,-7.246945,7.246945
...,...,...,...,...,...,...,...,...,...,...,...,...
48387,BUF,04:26,4.0,1.0,10.0,BAL 10,run,(4:26) 4-J.Cook up the middle to BAL 7 for 3 yards (14-K.Hamilton).,4.957158,-0.138851,7.110927,-7.110927
48389,BUF,04:21,4.0,2.0,7.0,BAL 7,run,(4:21) (Shotgun) 17-J.Allen right tackle to BAL 2 for 5 yards (92-N.Madubuike).,4.818307,-0.026381,7.084546,-7.084546
48391,BUF,04:17,4.0,3.0,2.0,BAL 2,run,(4:17) (Shotgun) 17-J.Allen up the middle to BAL 2 for no gain (29-A.Washington; 98-T.Jones).,4.791926,-1.825073,5.259472,-5.259472
48396,BAL,03:18,4.0,2.0,10.0,BAL 12,run,(3:18) (Shotgun) 8-L.Jackson scrambles right end pushed ob at BAL 20 for 8 yards (40-V.Miller).,-0.004842,0.321826,5.972833,-5.972833


In [200]:
game.loc[game['drive']==4, ['posteam','time','qtr','down','ydstogo','yrdln','play_type','desc','ep','epa','total_home_epa','total_away_epa']]

,posteam,time,qtr,down,ydstogo,yrdln,play_type,desc,ep,epa,total_home_epa,total_away_epa
48279,BUF,03:52,1.0,1.0,10.0,BUF 43,pass,(3:52) (Shotgun) 17-J.Allen pass short middle to 86-D.Kincaid to BAL 46 for 11 yards (40-M.Harrison).,2.558003,0.608715,3.384474,-3.384474
48280,BUF,03:09,1.0,1.0,10.0,BAL 46,run,(3:09) (Shotgun) 70-A.Anderson reported in as eligible. 4-J.Cook right guard to BAL 47 for -1 yards (95-T.Robinson).,3.166719,-0.810377,2.574097,-2.574097
48281,BUF,02:27,1.0,2.0,11.0,BAL 47,no_play,"(2:27) (Shotgun) 17-J.Allen pass short right to 1-C.Samuel to BAL 39 for 8 yards (49-C.Board; 2-N.Wiggins). PENALTY on BUF-73-D.Dawkins, Offensive Holding, 10 yards, enforced at BAL 47 - No Play.",2.356342,-0.623418,1.950680,-1.950680
48282,BUF,02:03,1.0,2.0,21.0,BUF 43,no_play,"(2:03) (Shotgun) PENALTY on BAL-99-O.Oweh, Neutral Zone Infraction, 5 yards, enforced at BUF 43 - No Play.",1.732925,0.224084,2.174763,-2.174763
48283,BUF,01:39,1.0,2.0,16.0,BUF 48,pass,(1:39) (Shotgun) 17-J.Allen sacked at BUF 47 for -1 yards (sack split by 53-K.Van Noy and 92-N.Madubuike).,1.957008,-1.071717,1.103046,-1.103046
48284,BUF,00:53,1.0,3.0,17.0,BUF 47,pass,(:53) (Shotgun) 17-J.Allen pass incomplete deep right to 88-D.Knox [53-K.Van Noy].,0.885292,-1.284603,-0.181556,0.181556
48285,BUF,00:48,1.0,4.0,17.0,BUF 47,punt,"(:48) 8-S.Martin punts 44 yards to BAL 9, Center-69-R.Ferguson, fair catch by 18-S.Sims.",-0.399311,0.507688,0.326132,-0.326132


In [201]:
# need to find drives with kickoffs and/or penalties, see if that explains the difference between summing epa and the total epa column